In [12]:


pip install yfinance pandas tabulate

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import yfinance as yf
import difflib
import pandas as pd
from tabulate import tabulate
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def safe_div(a, b):
    if a is None or b is None or b == 0:
        return None
    result = a / b
    # Flagging suspicious ratios
    if abs(result) > 1000:  # Ratio seems too extreme
        return None
    return result

# taking input
ticker = input("Enter the company ticker (e.g., HDFCBANK.NS): ").upper()
date = input("Enter the report date (YYYY-MM-DD): ")

# Fetching data 
try:
    comp = yf.Ticker(ticker)
    bs = comp.balance_sheet
    inc = comp.financials
    cf = comp.cashflow
except:
    print("Error: Could not fetch data for this ticker.")
    exit()

# fuzzy match function
def fuzzy_lookup(label, df):
    if df.empty:
        return None
    
    if label in df.index:
        return label
    
    # Keyword matching 
    key_words = {
        "Current Assets": ["current", "assets"],
        "Current Liabilities": ["current", "liabilities"],
        "Total Assets": ["total", "assets"],
        "Total Equity": ["equity", "stockholders", "shareholders"],
        "Total Liabilities": ["total", "liabilities"],
        "Long Term Debt": ["debt", "long"],
        "Total Debt": ["debt"],
        "Cash and Cash Equivalents": ["cash"],
        "Net Income": ["net", "income"],
        "Total Revenue": ["revenue", "total"],
        "Operating Income": ["operating", "income"],
        "Interest Expense": ["interest"],
        "Operating Cash Flow": ["operating", "cash", "flow"],
        "Free Cash Flow": ["free", "cash", "flow"],
        "Capital Expenditure": ["capital", "expenditure"],
        "Cost of Revenue": ["cost", "revenue"],
        "Gross Profit": ["gross", "profit"],
        "EBIT": ["ebit"],
        "EBITDA": ["ebitda"],
        "Inventory": ["inventory"],
        "Accounts Receivable": ["receivable", "accounts"],
        "Accounts Payable": ["payable", "accounts"],
        "Tax Provision": ["tax"]
    }
    
    # alternative names 
    alternatives = {
        "Current Assets": ["Total Current Assets", "Current Assets Total", "Total current assets"],
        "Current Liabilities": ["Total Current Liabilities", "Current Liabilities Total", "Total current liabilities"],
        "Total Assets": ["Total Assets", "Assets Total", "Total assets"],
        "Total Equity": ["Total Stockholders Equity", "Stockholders Equity", "Total Shareholders Equity", 
                        "Shareholders Equity", "Total stockholders equity", "Total Equity Gross Minority Interest"],
        "Total Liabilities": ["Total Liabilities Net Minority Interest", "Total liabilities"],
        "Long Term Debt": ["Long Term Debt And Capital Lease Obligation", "Long-term debt", "Long Term Debt"],
        "Cash and Cash Equivalents": ["Cash Cash Equivalents And Short Term Investments", "Cash And Cash Equivalents", 
                                    "Cash cash equivalents and short term investments", "Cash And Short Term Investments"],
        "Net Income": ["Net Income Common Stockholders", "Net income", "Net Income From Continuing And Discontinued Operation"],
        "Total Revenue": ["Total Revenue", "Revenue", "Total revenue"],
        "Operating Income": ["Operating Income", "Operating Revenue", "Operating income"],
        "Interest Expense": ["Interest Expense", "Interest expense", "Net Interest Income"],
        "Operating Cash Flow": ["Operating Cash Flow", "Cash Flow From Continuing Operating Activities"],
        "Free Cash Flow": ["Free Cash Flow", "Free cash flow"],
        "Capital Expenditure": ["Capital Expenditure", "Capital expenditure"],
        "Cost of Revenue": ["Cost Of Revenue", "Cost of revenue", "Total Expenses"],
        "Gross Profit": ["Gross Profit", "Gross profit"],
        "EBIT": ["EBIT", "Earnings Before Interest And Taxes"],
        "EBITDA": ["EBITDA", "Earnings Before Interest Taxes Depreciation And Amortization"],
        "Inventory": ["Inventory", "Total inventory"],
        "Accounts Receivable": ["Accounts Receivable", "Net Receivables", "Receivables"],
        "Accounts Payable": ["Accounts Payable", "Total Payables", "Payables"],
        "Tax Provision": ["Tax Provision", "Income Tax Expense", "Tax expense"]
    }
    
    if label in alternatives:
        for alt_name in alternatives[label]:
            if alt_name in df.index:
                return alt_name
    
    # Try keyword matching 
    if label in key_words:
        keywords = key_words[label]
        for idx in df.index:
            idx_lower = idx.lower()
            if all(keyword.lower() in idx_lower for keyword in keywords):
                return idx
    
    if label in key_words:
        keywords = key_words[label]
        for keyword in keywords:
            for idx in df.index:
                if keyword.lower() in idx.lower():
                    return idx
    
    # Try fuzzy matching with very low cutoff
    search_terms = [label]
    if label in alternatives:
        search_terms.extend(alternatives[label])
    
    for cutoff in [0.8, 0.6, 0.4, 0.2, 0.1]:
        for term in search_terms:
            matches = difflib.get_close_matches(term, df.index, n=1, cutoff=cutoff)
            if matches:
                return matches[0]
    

    label_words = label.lower().split()
    for word in label_words:
        if len(word) > 2: 
            for idx in df.index:
                if word in idx.lower():
                    return idx
    
    return None

# error handling
target_rows = {
    "Net Income": inc, "Total Revenue": inc, "EBIT": inc, "EBITDA": inc,
    "Interest Expense": inc, "Cost of Revenue": inc, "Gross Profit": inc,
    "Operating Income": inc, "Tax Provision": inc, "Total Assets": bs,
    "Total Equity": bs, "Total Liabilities": bs, "Current Assets": bs,
    "Current Liabilities": bs, "Long Term Debt": bs, "Total Debt": bs,
    "Cash and Cash Equivalents": bs, "Inventory": bs, "Accounts Receivable": bs,
    "Accounts Payable": bs, "Working Capital": bs, "Operating Cash Flow": cf,
    "Free Cash Flow": cf, "Capital Expenditure": cf,
}

values = {}
for label, df in target_rows.items():
    if df.empty:
        values[label] = None
        continue
        
    row = fuzzy_lookup(label, df)
    if row:
        try:
            if date in df.columns:
                values[label] = df.loc[row, date]
            else:
                
                values[label] = df.loc[row, df.columns[0]]
                
        
            if isinstance(values[label], str):
                try:
                    values[label] = float(values[label])
                except:
                    values[label] = None
        except:
            values[label] = None
    else:
        values[label] = None

#Calculate  values
if values["Working Capital"] is None and values["Current Assets"] is not None and values["Current Liabilities"] is not None:
    values["Working Capital"] = values["Current Assets"] - values["Current Liabilities"]

if values["Gross Profit"] is None and values["Total Revenue"] is not None and values["Cost of Revenue"] is not None:
    values["Gross Profit"] = values["Total Revenue"] - values["Cost of Revenue"]

if values["Total Debt"] is None and values["Long Term Debt"] is not None:
    values["Total Debt"] = values["Long Term Debt"]

# Compute ratios
profitability_ratios = {
    "Return on Equity (ROE)": safe_div(values["Net Income"], values["Total Equity"]),
    "Return on Assets (ROA)": safe_div(values["Net Income"], values["Total Assets"]),
    "Gross Profit Margin": safe_div(values["Gross Profit"], values["Total Revenue"]),
    "Operating Margin": safe_div(values["Operating Income"], values["Total Revenue"]),
    "Net Profit Margin": safe_div(values["Net Income"], values["Total Revenue"]),
    "EBIT Margin": safe_div(values["EBIT"], values["Total Revenue"]),
    "EBITDA Margin": safe_div(values["EBITDA"], values["Total Revenue"]),
}

# Calculate Quick Assets 
def calculate_quick_assets(current_assets, inventory):
    if current_assets is None:
        return None
    if inventory is None:
        return current_assets  # If no inventory data, use current assets
    
    # Data validation - inventory shouldn't exceed current assets
    if inventory > current_assets:
        return current_assets  # Use current assets if inventory seems wrong
    
    return current_assets - inventory

quick_assets = calculate_quick_assets(values["Current Assets"], values["Inventory"])

liquidity_ratios = {
    "Current Ratio": safe_div(values["Current Assets"], values["Current Liabilities"]),
    "Quick Ratio": safe_div(quick_assets, values["Current Liabilities"]),
    "Cash Ratio": safe_div(values["Cash and Cash Equivalents"], values["Current Liabilities"]),
    "Operating Cash Flow Ratio": safe_div(values["Operating Cash Flow"], values["Current Liabilities"]),
    "Working Capital Ratio": safe_div(values["Working Capital"], values["Total Revenue"]),
}

leverage_ratios = {
    "Debt to Equity": safe_div(values["Total Debt"], values["Total Equity"]),
    "Debt to Assets": safe_div(values["Total Debt"], values["Total Assets"]),
    "Total Liabilities to Equity": safe_div(values["Total Liabilities"], values["Total Equity"]),
    "Interest Coverage Ratio": safe_div(values["EBIT"], values["Interest Expense"]),
    "Debt Service Coverage": safe_div(values["Operating Cash Flow"], values["Total Debt"]),
}

efficiency_ratios = {
    "Asset Turnover": safe_div(values["Total Revenue"], values["Total Assets"]),
    "Equity Turnover": safe_div(values["Total Revenue"], values["Total Equity"]),
    "Receivables Turnover": safe_div(values["Total Revenue"], values["Accounts Receivable"]),
    "Inventory Turnover": safe_div(values["Cost of Revenue"], values["Inventory"]),
    "Payables Turnover": safe_div(values["Cost of Revenue"], values["Accounts Payable"]),
}

cash_flow_ratios = {
    "Operating CF to Net Income": safe_div(values["Operating Cash Flow"], values["Net Income"]),
    "Free Cash Flow to Revenue": safe_div(values["Free Cash Flow"], values["Total Revenue"]),
    "Cash Flow to Debt": safe_div(values["Operating Cash Flow"], values["Total Debt"]),
    "Capital Expenditure Ratio": safe_div(values["Capital Expenditure"], values["Total Revenue"]),
}

# Format function
def format_ratio(value):
    if value is None or pd.isna(value):
        return "N/A"
    elif abs(value) < 0.01:
        return f"{value:.4f}"
    elif abs(value) < 1:
        return f"{value:.3f}"
    else:
        return f"{value:.2f}"

# Creating DataFrame
all_ratios = []
categories = [
    ("PROFITABILITY RATIOS", profitability_ratios),
    ("LIQUIDITY RATIOS", liquidity_ratios), 
    ("LEVERAGE RATIOS", leverage_ratios),
    ("EFFICIENCY RATIOS", efficiency_ratios),
    ("CASH FLOW RATIOS", cash_flow_ratios)
]

for category_name, ratio_dict in categories:
    all_ratios.append([f"--- {category_name} ---", ""])
    for ratio_name, ratio_value in ratio_dict.items():
        all_ratios.append([ratio_name, format_ratio(ratio_value)])
    all_ratios.append(["", ""])

all_ratios = all_ratios[:-1]
df_ratios = pd.DataFrame(all_ratios, columns=["Financial Metric", "Value"])

# --- Display results
print(f"\n{'='*60}")
print(f"FINANCIAL RATIO ANALYSIS")
print(f"Company: {ticker}")
print(f"Report Date: {date}")
print(f"{'='*60}")

print(tabulate(df_ratios, headers="keys", tablefmt="fancy_grid", showindex=False))

# --- Summary statistics
print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}")

key_metrics = [
    ("ROE", format_ratio(profitability_ratios["Return on Equity (ROE)"])),
    ("ROA", format_ratio(profitability_ratios["Return on Assets (ROA)"])),
    ("Net Margin", format_ratio(profitability_ratios["Net Profit Margin"])),
    ("Current Ratio", format_ratio(liquidity_ratios["Current Ratio"])),
    ("Debt/Equity", format_ratio(leverage_ratios["Debt to Equity"])),
]

summary_df = pd.DataFrame(key_metrics, columns=["Key Metric", "Value"])
print(tabulate(summary_df, headers="keys", tablefmt="simple", showindex=False))


 FINANCIAL RATIO ANALYSIS
Company: HDFCBANK.NS
Report Date: 2023-03-31
╒══════════════════════════════╤═════════╕
│ Financial Metric             │ Value   │
╞══════════════════════════════╪═════════╡
│ --- PROFITABILITY RATIOS --- │         │
├──────────────────────────────┼─────────┤
│ Return on Equity (ROE)       │ 0.170   │
├──────────────────────────────┼─────────┤
│ Return on Assets (ROA)       │ 0.019   │
├──────────────────────────────┼─────────┤
│ Gross Profit Margin          │ 0.138   │
├──────────────────────────────┼─────────┤
│ Operating Margin             │ 1.00    │
├──────────────────────────────┼─────────┤
│ Net Profit Margin            │ 0.411   │
├──────────────────────────────┼─────────┤
│ EBIT Margin                  │ -0.0007 │
├──────────────────────────────┼─────────┤
│ EBITDA Margin                │ 0.019   │
├──────────────────────────────┼─────────┤
│                              │         │
├──────────────────────────────┼─────────┤
│ --- LIQUIDITY RATIOS --